# Lab 14: Periodic Data
In this lab we will learn how to work with Periodic data

Name:

In [ ]:
# initial imports
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table

# run the %matplotlib magic command to enable inline plotting
# in the current Notebook
%matplotlib inline

## Time Series data
All periodic analysis begins with Time Series Data. This data usually has three columns date, observation, observation error. The date is usually some type of Julian Date, although you may have to covert to Julian Date type before you do your analysis. The observation can be any type of varying quantity (e.g. magnitude, radial velocity, etc.).

In [ ]:
lightcurve = Table.read('data/SW_And_V.cur',format='ascii',names=('JD','Vmag','Vmag_err'))
plt.plot(lightcurve['JD']-2400000.5,lightcurve['Vmag'],'o')
plt.xlabel('MJD')
plt.ylabel('V Magnitude')
plt.gca().invert_yaxis() #Magnitudes are upside down

## Phase and Cycle
In order to work with periodic data we need to transform from date space to phase space. In order to do that, you need to know what the period of the data is. $$phase = \frac{JD - Max_JD}{period} - cycle$$
We often set the Max_JD (JD at the maximum of our periodic function) to 0, since we don't know it ahead of time. For AA Aql I know its period is 0.36145 days. The period needs to be in the same unit as our date, which in this case is also in days. The cycle is the number of full periods you have gone through since time = 0. It is just: $$cycle = floor(\frac{JD - Max_JD}{period})$$
In some cases, you may want to change the zero-point for your phases. For instance, if you want your Max_JD to be phase 0.5 instead of 0, you can add a 0.5 phase offset to your phase.


In [ ]:
def date2phase(date,period,ph_offset=0,max_jd=0):
    '''
    This function coverts julian dates into phases. 
    
    Arguments:
        date: A numpy array of dates in a julian date format (e.g JD, MJD, BJD, HJD, etc.)
        period: The period of the periodic signal in days.
    Keywords:
        ph_offset: Arbitrary phase offset between -1 and 1 (Default=0).
        max_jd: The epoch of maximum or epoch of zero phase in the same format as date (Default=0).
    returns:
        phase: A numpy array of phases with the same dimensions as date.
    '''
    phase = (date-max_jd)/period
    phase = (phase - np.floor(phase)) + ph_offset
    
    #Ensure that the phase stays between 0 and 1.
    phase[phase > 1] = phase[phase>1] -1
    phase[phase < 0] = phase[phase<0] +1    
    return phase

In [ ]:
lightcurve['Phase'] = date2phase(lightcurve['JD'],0.44226640)
plt.errorbar(lightcurve['Phase'],lightcurve['Vmag'],lightcurve['Vmag_err'],marker='o',linestyle='None')
plt.xlabel('Phase')
plt.ylabel('V Magnitude')
plt.gca().invert_yaxis() #Magnitudes are upside down

## Period Finding
In order to create a phased light curve, you need to know the period. There are many different algorithms available, but the most commonly used for variable stars is the Lomb-Scargle Periodogram. Lomb-Scargle basically fits a sinusoidal function to the data, and the it peaks, when that model fits the best.

For RR Lyrae Stars The minimum period is abou 0.1 days and the maximum period is 0.9 day. so let's set our limits there.

In [ ]:
from astropy.timeseries import LombScargle
min_freq = 1.11
max_freq = 3.3

frequency,power = LombScargle(lightcurve['JD'], lightcurve['Vmag'], lightcurve['Vmag_err'],nterms=1).autopower(
    minimum_frequency=min_freq, maximum_frequency=max_freq)

plt.figure(figsize=(10,6))
plt.subplot(211)
plt.plot(frequency, power,linewidth=0.5)
plt.xlabel("Frequency 1/day")
plt.ylabel("Lomb-Scargle Power")
plt.subplot(212)
plt.plot(1/frequency, power,linewidth=0.5)
plt.xlabel("Period (Days)")
plt.ylabel("Lomb-Scargle Power")
plt.show()

In [ ]:
best_frequency = frequency[np.argmax(power)]
best_period = 1/best_frequency
print("The Best Period is: {} and Best Frequency: {}".format(best_period,best_frequency))
lightcurve['Phase'] = date2phase(lightcurve['JD'],best_period)
plt.plot(lightcurve['Phase'],lightcurve['Vmag'],'o')
plt.xlabel('Phase')
plt.ylabel('V Magnitude')
plt.gca().invert_yaxis() #Magnitudes are upside down

## Fitting Splines
Because temporal data has gaps it is often useful to fit a spline to the data. This lets you fill in all the holes in your data using interpolation as well as average multiple data points per position. Splines can have different orders. The higher the order the more flexible the spline. If you let a spline get too flexible, you start fitting errors, so keep the order as low as you can. A spline object takes a series of x values, and returns y values.

In [ ]:
import scipy.interpolate as interp
def fitspline(phase,mag,error,order=3):
    #X values need to be sorted.
    sortind = np.argsort(phase)
    #Extend array by six on 0.0 end and 1.1 end
    extind = np.concatenate((sortind[-6:],sortind,sortind[0:6]))
    phase = phase[extind]
    phase[0:6] = phase[0:6] - 1.0
    phase[-6:] = phase[-6:] + 1.0
    #Do this with rest of arrays
    mag = mag[extind]
    error = error[extind]
    #Interpolater doesn't do well with nan's
    goodind = np.isfinite(mag)
    phase = phase[goodind]
    mag = mag[goodind]
    error = error[goodind]
    #Do the spline interpolation
    phasemag_sp = interp.UnivariateSpline(phase,mag,k=order,w=1.0/error)
    return phasemag_sp #This is a spline object

In [ ]:
#Fit spline
phasemag_sp = fitspline(lightcurve['Phase'],lightcurve['Vmag'],lightcurve['Vmag_err'],order=3)
#Order my data by phase
sortind = np.argsort(lightcurve['Phase'])
lightcurve['Phase_ph'] = lightcurve['Phase'][sortind]
lightcurve['Vmag_ph'] = lightcurve['Vmag'][sortind]
lightcurve['Vmag_err_ph'] = lightcurve['Vmag_err'][sortind]
plt.errorbar(lightcurve['Phase_ph'],lightcurve['Vmag_ph'],yerr=lightcurve['Vmag_err_ph'],fmt='.')
plt.plot(lightcurve['Phase_ph'],phasemag_sp(lightcurve['Phase_ph']))
plt.xlabel('Phase')
plt.ylabel('V Magnitude')
plt.gca().invert_yaxis() #Magnitudes are upside down